# 07. Model Deployment

In [ ]:
!pip install ipykernel>=5.1.2
!pip install pydeck


In [ ]:
conda install -c conda-forge shap

In [1]:
import streamlit as st
import pandas as pd
import shap
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.ensemble import RandomForestRegressor

In [2]:
import numpy as np

In [28]:
import streamlit as st
import pickle


In [31]:
model = pickle.load(open('RF_price_predicting_model.pkl','rb'))

In [3]:
st.write("""
# VehiCALC
This app predicts the **Used Cars Price**!
""")
st.write('---')

2022-01-01 15:16:59.545 
  command:

    streamlit run /opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [6]:
# Loads the Used Cars Dataset
cars_final = pd.read_csv('/users/alfon/Desktop/Master/TFM/CSV/03.cars_final_def.csv')
cars_final = cars_final.drop(['Version', 'ZIP'], axis=1)
X= cars_final[cars_final.columns[:-1]]
y= cars_final[cars_final.columns[-1]]

In [8]:
# Sidebar
# Header of Specify Input Parameters
st.sidebar.header('Specify Input Parameters')

DeltaGenerator(_root_container=1, _provided_cursor=None, _parent=DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None), _block_type=None, _form_data=None)

In [10]:
def user_input_features():
    YEAR = st.sidebar.slider('Year', int(X.Year.min()), int(X.Year.max()), int(X.Year.mean()))
    KMS = st.sidebar.slider('Kms', int(X.Kms.min()), int(X.Kms.max()), int(X.Kms.mean()))
    HP = st.sidebar.slider('Hp', int(X.Hp.min ()), int(X.Hp.max()), int(X.Hp.mean()))
    CONS = st.sidebar.slider('Fuel_cons', int(X.Fuel_cons.min()), int(X.Fuel_cons.max()), int(X.Fuel_cons.mean()))
    DOORS = st.sidebar.slider('Doors', int(X.Doors.min()), int(X.Doors.max()), int(X.Doors.mean()))
    TRANSMISSION = st.sidebar.selectbox('Gear_type', X.Gear_type.unique())
    SELLER = st.sidebar.selectbox('Seller', X.Seller.unique())
    
    

    BRAND = st.sidebar.selectbox('Brand', np.sort(cars_final.Brand.unique()), index=0, help='Choose car brand')
    MODEL = st.sidebar.selectbox('Model', np.sort(cars_final[cars_final.Brand == BRAND].Model.unique()), index=0, help='Models available for the selected brand')
    TYPE = st.sidebar.selectbox('Type', cars_final.Type.unique(), index=0)
    PROVINCE = st.sidebar.selectbox('Province', cars_final.Province.unique(), index=0)
    COLOUR = st.sidebar.selectbox('Colour', cars_final.Colour.unique(), index=0)
    FUEL = st.sidebar.selectbox('Fuel_type', cars_final.Fuel_type.unique(), index=0)
        

        
    
    
    
    data = {'Brand': BRAND,
            'Model': MODEL,
            'Type': TYPE,
            'Year': YEAR,
            'Kms': KMS,
            'Hp': HP,
            'Gear_type': TRANSMISSION,
            'Fuel_type': FUEL,
            'Fuel_cons': CONS,
            'Doors': DOORS,
            'Colour': COLOUR,
            'Province': PROVINCE,
            'Seller': SELLER}
    
    features = pd.DataFrame(data, index=[0])
    return features

df_frontend = user_input_features()

In [12]:
df_frontend.head()

,Brand,Model,Type,Year,Kms,Hp,Gear_type,Fuel_type,Fuel_cons,Doors,Colour,Province,Seller
0,Abarth,124 Spider,small,2013,106781,172,Manual,Diesel,6,4,Beige,Barcelona,Dealer


In [14]:
X.sample(1)

,Brand,Model,Type,Year,Kms,Hp,Gear_type,Fuel_type,Fuel_cons,Doors,Colour,Province,Seller
9267,MINI,Cooper D,small,2019,61632,116,Manual,Diesel,3.5,5,White,Córdoba,Dealer


In [15]:
df = pd.concat([df_frontend, X], axis=0).reset_index().drop('index', axis=1)

In [16]:
for col in ['Brand','Gear_type', 'Fuel_type','Type','Seller']:
    df[col] = df[col].astype('category')

In [17]:
df = pd.get_dummies(data=df,columns=['Gear_type','Fuel_type','Type','Seller'])

In [18]:
from category_encoders import TargetEncoder

In [19]:
encoder = TargetEncoder()

In [20]:
cols_to_encode = ['Brand','Model', 'Colour', 'Province']
cols_encoded = list(map(lambda c: c + '_encoded', cols_to_encode))

df[cols_encoded] = encoder.fit_transform(df[cols_to_encode], df.Year)

In [33]:
df.drop(['Brand','Model', 'Colour', 'Province'], axis = 1, inplace = True)



In [34]:
df_pred = df[:1]

In [35]:
df_pred

,Year,Kms,Hp,Fuel_cons,Doors,Gear_type_Automatic,Gear_type_Manual,Fuel_type_CNG,Fuel_type_Diesel,Fuel_type_Electric,...,Type_sedan,Type_small,Type_suv,Type_van,Seller_Dealer,Seller_Private,Brand_encoded,Model_encoded,Colour_encoded,Province_encoded
0,2013,106781,172,6.0,4,0,1,0,1,0,...,0,1,0,0,1,0,2016.979695,2017.052632,2012.451389,2012.972741


In [36]:
# Print specified input parameters
st.header('Specified Input parameters')
st.write(df)
st.write('---')

In [37]:
# Apply Model to Make Prediction
prediction = model.predict(df_pred)

In [ ]:
st.header('Prediction of MEDV')
st.write(prediction)
st.write('---')